In [ ]:
%load_ext autoreload
%autoreload 2
import os, sys

import numpy as np

from matplotlib import pyplot as plt
from matplotlib_settings import set_plot_settings, reset_plot_settings

# Set the plot settings
set_plot_settings()

# import global variables
from utils_motor_global import *

from utils_motor_sigproc import compute_overall_mean, compute_overall_std_dev


ROOT_LOAD_DIR = f'{REC_DIR}/6_truncate'
ROOT_SAVE_DIR = f'{REC_DIR}/7_channel_band_mu_sigma'

In [ ]:
""" initialize """
session_lens = []

session_spect_mus = []
session_lmp_mus   = []
session_lfs_mus   = []
session_beta_mus  = []
session_lga_mus   = []
session_hga_mus   = []

session_spect_sigmas = []
session_lmp_sigmas   = []
session_lfs_sigmas   = []
session_beta_sigmas  = []
session_lga_sigmas   = []
session_hga_sigmas   = []

In [ ]:
for key in SESSION_KEYS:

    load_dir = f'{ROOT_LOAD_DIR}/{key}'
    if not os.path.exists(load_dir):
        continue
    print(f'processing {key}...')

    """ load data """
    good_chs       = np.load(f'{load_dir}/good_channels_{key}.npy')
    spect          = np.load(f'{load_dir}/spect_{key}.npy') # channel * time * freq
    spect_t        = np.load(f'{load_dir}/spect_t_{key}.npy')
    spect_f        = np.load(f'{load_dir}/spect_f_{key}.npy')
    beta_fidxs     = np.load(f'{load_dir}/beta_fidxs_{key}.npy')
    lga_fidxs      = np.load(f'{load_dir}/lga_fidxs_{key}.npy')
    hga_fidxs      = np.load(f'{load_dir}/hga_fidxs_{key}.npy')
    lmp_data       = np.load(f'{load_dir}/lmp_{key}.npy')

    """ compute band power """
    spect_beta = np.sum(spect[:,:,beta_fidxs], axis=2)
    spect_lga  = np.sum(spect[:,:, lga_fidxs], axis=2)
    spect_hga  = np.sum(spect[:,:, hga_fidxs], axis=2)

    """ compute per session mu, std dev """
    session_spect_mu = np.full((NCH, spect.shape[2]), np.nan)
    session_lmp_mu   = np.full((NCH, ), np.nan)
    session_lfs_mu   = np.full((NCH, ), np.nan)
    session_beta_mu  = np.full((NCH, ), np.nan)
    session_lga_mu   = np.full((NCH, ), np.nan)
    session_hga_mu   = np.full((NCH, ), np.nan)

    session_spect_sigma = np.full((NCH, spect.shape[2]), np.nan)
    session_lmp_sigma   = np.full((NCH, ), np.nan)
    session_lfs_sigma   = np.full((NCH, ), np.nan)
    session_beta_sigma  = np.full((NCH, ), np.nan)
    session_lga_sigma   = np.full((NCH, ), np.nan)
    session_hga_sigma   = np.full((NCH, ), np.nan)

    session_spect_mu[good_chs,:] = np.mean(spect, axis=1)
    session_lmp_mu  [good_chs]   = np.mean(lmp_data, axis=1)
    session_lfs_mu  [good_chs]   = np.mean(spect[:,:,0], axis=1)
    session_beta_mu [good_chs]   = np.mean(spect_beta, axis=1)
    session_lga_mu  [good_chs]   = np.mean(spect_lga, axis=1)
    session_hga_mu  [good_chs]   = np.mean(spect_hga, axis=1)

    session_spect_sigma[good_chs,:] = np.std(spect, axis=1)
    session_lmp_sigma  [good_chs]   = np.std(lmp_data, axis=1)
    session_lfs_sigma  [good_chs]   = np.std(spect[:,:,0], axis=1)
    session_beta_sigma [good_chs]   = np.std(spect_beta, axis=1)
    session_lga_sigma  [good_chs]   = np.std(spect_lga, axis=1)
    session_hga_sigma  [good_chs]   = np.std(spect_hga, axis=1)

    """ append..  """
    session_spect_mus.append(session_spect_mu)
    session_lmp_mus.append(session_lmp_mu  )
    session_lfs_mus.append(session_lfs_mu  )
    session_beta_mus.append(session_beta_mu )
    session_lga_mus.append(session_lga_mu  )
    session_hga_mus.append(session_hga_mu  )

    session_spect_sigmas.append(session_spect_sigma)
    session_lmp_sigmas.append(session_lmp_sigma  )
    session_lfs_sigmas.append(session_lfs_sigma  )
    session_beta_sigmas.append(session_beta_sigma )
    session_lga_sigmas.append(session_lga_sigma  )
    session_hga_sigmas.append(session_hga_sigma  )

    session_lens.append(len(spect_t))

# check that all spectrograms have same frequency bins
for s0, s1 in zip(session_spect_sigmas[:-1], session_spect_sigmas[1:]):
    assert s0.shape == s1.shape

In [ ]:
# convert to arrays
session_spect_mus = np.array(session_spect_mus)
session_lmp_mus   = np.array(session_lmp_mus  )
session_lfs_mus   = np.array(session_lfs_mus  )
session_beta_mus  = np.array(session_beta_mus )
session_lga_mus   = np.array(session_lga_mus  )
session_hga_mus   = np.array(session_hga_mus  )

session_spect_sigmas = np.array(session_spect_sigmas)
session_lmp_sigmas   = np.array(session_lmp_sigmas  )
session_lfs_sigmas   = np.array(session_lfs_sigmas  )
session_beta_sigmas  = np.array(session_beta_sigmas )
session_lga_sigmas   = np.array(session_lga_sigmas  )
session_hga_sigmas   = np.array(session_hga_sigmas  )
session_lens         = np.array(session_lens)

In [ ]:
""" compute all-session mean """
spect_mu = np.full((NCH, spect.shape[2]), np.nan)
lmp_mu   = np.full((NCH, ), np.nan)
lfs_mu   = np.full((NCH, ), np.nan)
beta_mu  = np.full((NCH, ), np.nan)
lga_mu   = np.full((NCH, ), np.nan)
hga_mu   = np.full((NCH, ), np.nan)

for ch in range(NCH):
    for fidx in range(len(spect_f)):
        spect_mu[ch, fidx] = compute_overall_mean(session_spect_mus[:,ch,fidx],
                                                        session_lens)
    lmp_mu[ch]  = compute_overall_mean(session_lmp_mus[:,ch], session_lens)
    lfs_mu[ch]  = compute_overall_mean(session_lfs_mus[:,ch], session_lens)
    beta_mu[ch] = compute_overall_mean(session_beta_mus[:,ch], session_lens)
    lga_mu[ch]  = compute_overall_mean(session_lga_mus[:,ch], session_lens)
    hga_mu[ch]  = compute_overall_mean(session_hga_mus[:,ch], session_lens)

In [ ]:
""" compute all-session SD """
spect_sigma = np.full((NCH, spect.shape[2]), np.nan)
lmp_sigma   = np.full((NCH, ), np.nan)
lfs_sigma   = np.full((NCH, ), np.nan)
beta_sigma  = np.full((NCH, ), np.nan)
lga_sigma   = np.full((NCH, ), np.nan)
hga_sigma   = np.full((NCH, ), np.nan)

for ch in range(NCH):
    for fidx in range(len(spect_f)):
        spect_sigma[ch, fidx] = compute_overall_std_dev(session_spect_sigmas[:,ch,fidx],
                                                        session_lens)
    lmp_sigma[ch] = compute_overall_std_dev(session_lmp_sigmas[:,ch], session_lens)
    lfs_sigma[ch] = compute_overall_std_dev(session_lfs_sigmas[:,ch], session_lens)
    beta_sigma[ch] = compute_overall_std_dev(session_beta_sigmas[:,ch], session_lens)
    lga_sigma[ch] = compute_overall_std_dev(session_lga_sigmas[:,ch], session_lens)
    hga_sigma[ch] = compute_overall_std_dev(session_hga_sigmas[:,ch], session_lens)

In [ ]:
""" save sigmas """
save_data_dir = f'{ROOT_SAVE_DIR}'
if not os.path.exists(save_data_dir):
    os.makedirs(save_data_dir)

np.save(f'{save_data_dir}/spect_mu.npy',spect_mu)
np.save(f'{save_data_dir}/lmp_mu.npy',  lmp_mu)
np.save(f'{save_data_dir}/lfs_mu.npy',  lfs_mu)
np.save(f'{save_data_dir}/beta_mu.npy', beta_mu)
np.save(f'{save_data_dir}/lga_mu.npy',  lga_mu)
np.save(f'{save_data_dir}/hga_mu.npy',  hga_mu)

np.save(f'{save_data_dir}/spect_sigma.npy',spect_sigma)
np.save(f'{save_data_dir}/lmp_sigma.npy',  lmp_sigma)
np.save(f'{save_data_dir}/lfs_sigma.npy',  lfs_sigma)
np.save(f'{save_data_dir}/beta_sigma.npy', beta_sigma)
np.save(f'{save_data_dir}/lga_sigma.npy',  lga_sigma)
np.save(f'{save_data_dir}/hga_sigma.npy',  hga_sigma)

In [ ]:
# plot_data = lmp_sigma
# plot_data = beta_sigma
# plot_data = lga_sigma
# plot_data = hga_sigma

# q_cut = 0.03
# vmin = np.nanquantile(plot_data, q_cut)
# vmax = np.nanquantile(plot_data, 1-q_cut)
# plt.imshow(plot_data.reshape(16, -1), vmin=vmin, vmax=vmax)

In [ ]:
# headers = ['spect', 'lmp', 'beta', 'lga', 'hga']
# for key in SESSION_KEYS:
#     for header in headers:

#         # fn = f'{header}_mu.npy'
#         fn = f'{header}_sigma.npy'

#         dir0 = f'./combined_preprocessed_v3/2_space_freq_moments'
#         dir1 = f'./recording_postprocess_v3/7_channel_band_mu_sigma'

#         X0 = np.load(f'{dir0}/{fn}')
#         X1 = np.load(f'{dir1}/{fn}')
#         assert np.allclose(X0, X1, equal_nan=True)